In [1]:
import glob
import os
import unicodedata
import string
import torch
import random

In [7]:
nizami = open("nizami_yg.txt", 'r')
nizami = nizami.readlines()

In [16]:
for index, line in enumerate(nizami):
    if line[0] == '■':
        mk = line
        print(index)
        break

SyntaxError: invalid character '■' (U+25A0) (3787832125.py, line 7)

In [13]:
nizami[:] = [val for val in nizami if val[0] != mk]
# nizami[:] = [val for val in nizami if val != '\n']
# nizami[:] = [val for val in nizami if !(val == '\n' and )]

In [4]:
nizami_ = []
for index, line in enumerate(nizami):
    # if line == '\n' and nizami[index+1][0] in "0123456789":
    #     continue
    if line[0] in "0123456789" and len(line) <= 15:
        continue
    
    if line == '\n' and nizami[index+2] == '\n':
        # print('hi')
        continue
    else:
        nizami_.append(line)

nizami_[640:680]

['Hökmünə Tanrınm xalqın həyandır.\n',
 'Feyzi qəbul edər öz Tanrısmdan,\n',
 'Sonra bəndələrə yetirər haman.\n',
 'O verdi cahana bəxti, işrəti,\n',
 'Görüm bəxtiyarlıq olsun neməti.\n',
 'Səadət ömründən heç əskilməsin,\n',
 'Şahlığı dünyada zaval bilməsin.\n',
 'Görüm övladmm nəsilbənəsil\n',
 'Səadət ulduzu gülsün müttəsil.\n',
 'İki gözəl sübhün nuruyla par-par\n',
 'Gün kimi parlasın ulu tacidar.\n',
 'Uca taxt sahibi o şahzadalar\n',
 'Biri cahangirdir, biri cahandar.\n',
 'Biri Firiduntək bilikdə dərin,\n',
 'Biri Keyxosrovtək at çapar yeyin.\n',
 'Taxtından adına ellər agahdır,\n',
 'Nüsrətəddin Məlik Məhəmməd şahdır.\n',
 'Rasidlər deyir ki, qeyd etmiş fələk,\n',
 'Fələkəddin Əhməd adlansın gərək51.\n',
 'Birinin açarı nüsrətdən çatdı,\n',
 'Birini zəfərdən fələk yaratdı.\n',
 'Birinə tərbiyə nüsrət veribdir,\n',
 'Fələk o birinə qüdrət veribdir.\n',
 'Biri xoşbəxt oldu nüsrətdən gerçək,\n',
 'Birinə dörd payə taxt oldu fələk52.\n',
 'Mina çarx altında şahın gözləri\n',
 'O i

In [15]:
f = open('nizami_yg_.txt', 'w')
for line in nizami_:
    f.write(line)
    
f.close()

In [161]:
lenn = 0
for line in nizami_:
    lenn += len(line)
lenn

407296

In [ ]:
all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)

def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

In [ ]:
# Build the category_lines dictionary, a list of names per language
category_lines = {}
all_categories = []

def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines]

for filename in glob.glob('data/names/*.txt'):
    category = os.path.splitext(os.path.basename(filename))[0]
    all_categories.append(category)
    lines = readLines(filename)
    category_lines[category] = lines

n_categories = len(all_categories)

In [ ]:
# Find letter index from all_letters, e.g. "a" = 0
def letterToIndex(letter):
    return all_letters.find(letter)

# Just for demonstration, turn a letter into a <1 x n_letters> Tensor
def letterToTensor(letter):
    tensor = torch.zeros(1, n_letters)
    tensor[0][letterToIndex(letter)] = 1
    return tensor

# Turn a line into a <line_length x 1 x n_letters>,
# or an array of one-hot letter vectors
def lineToTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li, letter in enumerate(line):
        tensor[li][0][letterToIndex(letter)] = 1
    return tensor

print(letterToTensor('J'))

print(lineToTensor('Jones').size())

In [ ]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.h2o = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.h2o(hidden)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)
    

class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers = 1):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_size=input_size,
                            hidden_size=hidden_size,
                            num_layers = num_layers,
                            batch_first=True)
        
        self.linear = nn.Linear(hidden_size, output_size)
    
    def forward(self, x, hidden):
        out, hidden = self.lstm(x, hidden)
        out = out.view(-1, self.hidden_size)
        out = self.linear(out)
        return out, hidden
    
    def init_hidden(self, batch_size=1):
        h0 = torch.zeros(self.num_layers, self.hidden_size)
        c0 = torch.zeros(self.num_layers, self.hidden_size)
        return h0, c0

n_hidden = 128
rnn = RNN(n_letters, n_hidden, n_categories)
lstm = LSTM(n_letters, n_hidden, n_categories)

In [ ]:
input = lineToTensor('Albert')
h0, c0 = lstm.init_hidden()

output, (h, c) = lstm(input[0], (h0, c0))

def categoryFromOutput(output):
    top_n, top_i = output.topk(1)
    category_i = top_i[0].item()
    return all_categories[category_i], category_i

print(categoryFromOutput(output))

In [ ]:


def randomChoice(l):
    return l[random.randint(0, len(l) - 1)]

def randomTrainingExample():
    category = randomChoice(all_categories)
    line = randomChoice(category_lines[category])
    category_tensor = torch.tensor([all_categories.index(category)], dtype=torch.long)
    line_tensor = lineToTensor(line)
    return category, line, category_tensor, line_tensor

In [ ]:
criterion = nn.NLLLoss()

learning_rate = 0.005 # If you set this too high, it might explode. If too low, it might not learn

def train(category_tensor, line_tensor):
    hidden = rnn.initHidden()

    rnn.zero_grad()

    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)

    loss = criterion(output, category_tensor)
    loss.backward()

    # Add parameters' gradients to their values, multiplied by learning rate
    for p in rnn.parameters():
        p.data.add_(p.grad.data, alpha=-learning_rate)

    return output, loss.item()

In [ ]:
import time
import math

n_iters = 100000
print_every = 5000
plot_every = 1000


# Keep track of losses for plotting
current_loss = 0
all_losses = []

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

start = time.time()

for iter in range(1, n_iters + 1):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    output, loss = train(category_tensor, line_tensor)
    current_loss += loss

    # Print ``iter`` number, loss, name and guess
    if iter % print_every == 0:
        guess, guess_i = categoryFromOutput(output)
        correct = '✓' if guess == category else '✗ (%s)' % category
        print('%d %d%% (%s) %.4f %s / %s %s' % (iter, iter / n_iters * 100, timeSince(start), loss, line, guess, correct))

    # Add current loss avg to list of losses
    if iter % plot_every == 0:
        all_losses.append(current_loss / plot_every)
        current_loss = 0

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

plt.figure()
plt.plot(all_losses)

In [ ]:
# Keep track of correct guesses in a confusion matrix
confusion = torch.zeros(n_categories, n_categories)
n_confusion = 10000

# Just return an output given a line
def evaluate(line_tensor):
    hidden = rnn.initHidden()

    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)

    return output

# Go through a bunch of examples and record which are correctly guessed
for i in range(n_confusion):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    output = evaluate(line_tensor)
    guess, guess_i = categoryFromOutput(output)
    category_i = all_categories.index(category)
    confusion[category_i][guess_i] += 1

# Normalize by dividing every row by its sum
for i in range(n_categories):
    confusion[i] = confusion[i] / confusion[i].sum()

# Set up plot
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(confusion.numpy())
fig.colorbar(cax)

# Set up axes
ax.set_xticklabels([''] + all_categories, rotation=90)
ax.set_yticklabels([''] + all_categories)

# Force label at every tick
ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

# sphinx_gallery_thumbnail_number = 2
plt.show()

In [ ]:
def predict(input_line, n_predictions=3):
    print('\n> %s' % input_line)
    with torch.no_grad():
        output = evaluate(lineToTensor(input_line))

        # Get top N categories
        topv, topi = output.topk(n_predictions, 1, True)
        predictions = []

        for i in range(n_predictions):
            value = topv[0][i].item()
            category_index = topi[0][i].item()
            print('(%.2f) %s' % (value, all_categories[category_index]))
            predictions.append([value, all_categories[category_index]])

predict('Meraj')
